In [2]:
import os
import numpy as np
import pandas as pd
from nbeats_forecast import NBeats
from sklearn.metrics import mean_squared_error

In [5]:
file = "C:/Users/gurpr/Videos/BetaSci/Rossmann_Store1_Data_FULL_NBeats.csv"
pwd = os.getcwd()
os.chdir(os.path.dirname(file))
data = pd.read_csv(os.path.basename(file))
data = data.values        #univariate time series data of shape nx1 (numpy array)

#nparray = data["Sale"].to_numpy()
#model = NBeats(data=data, period_to_forecast=194, train_percent=0.75)
model = NBeats(data=data, period_to_forecast=194, train_percent=0.75)
model.fit()
forecast = model.predict()

| N-Beats
| --  Stack Generic (#0) (share_weights_in_stack=False)
     | -- GenericBlock(units=128, thetas_dim=7, backcast_length=582, forecast_length=194, share_thetas=False) at @1745205044992
     | -- GenericBlock(units=128, thetas_dim=7, backcast_length=582, forecast_length=194, share_thetas=False) at @1745205045808
     | -- GenericBlock(units=128, thetas_dim=7, backcast_length=582, forecast_length=194, share_thetas=False) at @1745205046096
| --  Stack Generic (#1) (share_weights_in_stack=False)
     | -- GenericBlock(units=128, thetas_dim=8, backcast_length=582, forecast_length=194, share_thetas=False) at @1745205044512
     | -- GenericBlock(units=128, thetas_dim=8, backcast_length=582, forecast_length=194, share_thetas=False) at @1745205046288
     | -- GenericBlock(units=128, thetas_dim=8, backcast_length=582, forecast_length=194, share_thetas=False) at @1745205046384
grad_step = 000210, tr_loss = 0.001082, te_loss = 0.479394
grad_step = 000240, tr_loss = 0.000742, te_loss = 0

In [6]:
y_test = data.ta
rmse = mean_squared_error(y_test, forecast , squared=False)

AttributeError: 'numpy.ndarray' object has no attribute 'ta'

In [7]:
print(forecast)

[[6530.8955]
 [6009.882 ]
 [5330.859 ]
 [4571.4707]
 [4550.2505]
 [4987.7583]
 [5102.5195]
 [6271.6387]
 [7483.38  ]
 [7812.347 ]
 [7503.7   ]
 [7443.582 ]
 [7589.7065]
 [7714.53  ]
 [8984.557 ]
 [8499.685 ]
 [5433.928 ]
 [4949.632 ]
 [6208.8457]
 [6463.481 ]
 [4144.8657]
 [4249.9014]
 [5081.402 ]
 [5730.4   ]
 [5101.282 ]
 [4550.5264]
 [4053.1577]
 [3838.2644]
 [4364.6875]
 [5080.3823]
 [5084.009 ]
 [4670.44  ]
 [4221.135 ]
 [4171.6074]
 [4801.668 ]
 [4201.382 ]
 [3638.6233]
 [3402.7778]
 [3537.459 ]
 [3550.7266]
 [4366.493 ]
 [4666.265 ]
 [4837.196 ]
 [4397.3716]
 [4635.7065]
 [5269.381 ]
 [5336.    ]
 [5485.7007]
 [5215.653 ]
 [4684.559 ]
 [4429.54  ]
 [4690.9053]
 [4634.437 ]
 [4183.2446]
 [3441.771 ]
 [3440.882 ]
 [3734.335 ]
 [4366.3384]
 [4858.8994]
 [4609.0513]
 [4442.414 ]
 [4344.8115]
 [4264.7266]
 [4533.24  ]
 [4561.274 ]
 [3992.4868]
 [3777.161 ]
 [3808.6995]
 [3986.0476]
 [4438.153 ]
 [5103.775 ]
 [5756.3813]
 [5600.8467]
 [5546.0977]
 [4712.059 ]
 [4736.1943]
 [4868.06  ]

In [ ]:
out = np.array(forecast, dtype=int)
out.round()
print(out)
np.savetxt(r"C:\Users\gurpr\Videos\BetaSci\NBeats_Pred194Using0.75.csv", out, delimiter=",", fmt='%.0i')